<h1>Add Trajectory Summary to Dataset.ipynb</h1>

In [1]:
import pandas as pd
from datetime import timedelta
import pyarrow.parquet as pq

In [2]:
challenge_set = pd.read_csv('../challenge_set.csv')

challenge_set['date'] = pd.to_datetime(challenge_set['date'])

display(challenge_set)

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,arrival_time,aircraft_type,wtc,airline,flight_duration,taxiout_time,flown_distance,tow
0,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01T13:46:00Z,2022-01-01T15:04:56Z,A320,M,a73f82288988b79be490c6322f4c32ed,61,18,321,54748.000000
1,248760618,2022-01-01,f6f610e73002b8892a239a81321f7f1d,LEBL,Barcelona,ES,KMIA,Miami,US,2022-01-01T09:55:00Z,2022-01-01T19:37:56Z,B772,H,5543e4dc327359ffaf5b9c0e6faaf0e1,570,13,4193,185441.000000
2,248753824,2022-01-01,139670936660762c230ca92556ba842b,ESSA,Stockholm Arlanda,SE,KORD,Chicago O'Hare,US,2022-01-01T09:39:00Z,2022-01-01T19:08:13Z,A333,H,8be5c854fd664bcb97fb543339f74770,554,15,3770,230396.000000
3,248753852,2022-01-01,509dc61bb54fbab0e5406067c95603e2,LSZH,Zurich,CH,KPHL,Philadelphia,US,2022-01-01T11:04:00Z,2022-01-01T19:32:13Z,B788,H,5543e4dc327359ffaf5b9c0e6faaf0e1,497,11,3607,157615.000000
4,248755934,2022-01-01,d0610d000dcf26b1d7bba8103ecc393d,EIDW,Dublin,IE,EGLL,London Heathrow,GB,2022-01-01T12:36:00Z,2022-01-01T13:44:32Z,A21N,M,a73f82288988b79be490c6322f4c32ed,55,14,305,70318.447226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369008,258058361,2022-12-31,85ee68e5b7b5acf24ba00d1318eca1e8,LFPG,Paris Charles de Gaulle,FR,KMIA,Miami,US,2022-12-31T09:38:00Z,2022-12-31T19:03:48Z,B788,H,5543e4dc327359ffaf5b9c0e6faaf0e1,545,21,4046,163438.000000
369009,258071247,2022-12-31,570cf7d5ebbd691bcba63e7466607da7,LTFM,iGA Istanbul,TR,EDDB,Berlin Brandenburg,DE,2022-12-31T09:27:00Z,2022-12-31T12:29:41Z,A21N,M,6351ec1b849adacc0cbb3b1313d8d39b,158,25,946,78707.000000
369010,258059152,2022-12-31,5a7e43e4f981539ae3d3b1cb31591b7c,EDDL,Dusseldorf,DE,EIDW,Dublin,IE,2022-12-31T09:52:00Z,2022-12-31T11:41:51Z,A320,M,a73f82288988b79be490c6322f4c32ed,99,11,522,62942.750000
369011,258072276,2022-12-31,a1c078516f9f9e90cacec61854cad45b,LFPG,Paris Charles de Gaulle,FR,EIDW,Dublin,IE,2022-12-31T09:37:00Z,2022-12-31T11:13:10Z,A21N,M,a73f82288988b79be490c6322f4c32ed,84,12,466,72611.161024


In [3]:
def calculate_stat(
    dataframe: pd.DataFrame, 
    group_by_column: str, 
    target_column: str,
    stat_type: str
) -> pd.Series | None:
    result = None
    if stat_type == 'median':
        result = dataframe.groupby(group_by_column)[target_column].median()
    elif stat_type == 'mean':
        result = dataframe.groupby(group_by_column)[target_column].mean()
    return result

In [4]:
def summarize_trajectories(
    trajectory_df: pd.DataFrame
) -> pd.DataFrame:
    latitude_median = calculate_stat(dataframe=trajectory_df, group_by_column='flight_id', target_column='latitude', stat_type='median')
    longitude_median = calculate_stat(dataframe=trajectory_df, group_by_column='flight_id', target_column='longitude', stat_type='median')
    altitude_mean = calculate_stat(dataframe=trajectory_df, group_by_column='flight_id', target_column='altitude', stat_type='mean')
    groundspeed_mean = calculate_stat(dataframe=trajectory_df, group_by_column='flight_id', target_column='groundspeed', stat_type='mean')
    track_mean = calculate_stat(dataframe=trajectory_df, group_by_column='flight_id', target_column='track', stat_type='mean')
    vertical_rate_mean = calculate_stat(dataframe=trajectory_df, group_by_column='flight_id', target_column='vertical_rate', stat_type='mean')
    track_unwrapped_mean = calculate_stat(dataframe=trajectory_df, group_by_column='flight_id', target_column='track_unwrapped', stat_type='mean')
    u_component_of_wind_mean = calculate_stat(dataframe=trajectory_df, group_by_column='flight_id', target_column='u_component_of_wind', stat_type='mean')
    v_component_of_wind_mean = calculate_stat(dataframe=trajectory_df, group_by_column='flight_id', target_column='v_component_of_wind', stat_type='mean')
    temperature_mean = calculate_stat(dataframe=trajectory_df, group_by_column='flight_id', target_column='temperature', stat_type='mean')
    specific_humidity_mean = calculate_stat(dataframe=trajectory_df, group_by_column='flight_id', target_column='specific_humidity', stat_type='mean')

    trajactory_summary_df = pd.DataFrame({
        "latitude_median": latitude_median,
        "longitude_median": longitude_median,
        "altitude_mean": altitude_mean,
        "groundspeed_mean": groundspeed_mean,
        "track_mean": track_mean,
        "vertical_rate_mean": vertical_rate_mean,
        "track_unwrapped_mean": track_unwrapped_mean,
        "u_component_of_wind_mean": u_component_of_wind_mean,
        "v_component_of_wind_mean": v_component_of_wind_mean,
        "temperature_mean": temperature_mean,
        "specific_humidity_mean": specific_humidity_mean
    })
    trajactory_summary_df = trajactory_summary_df.reset_index()
    trajactory_summary_df.rename(columns={'index': 'flight_id'}, inplace=True)
    trajactory_summary_df['flight_id'] = trajactory_summary_df['flight_id'].astype('int64')
    
    return trajactory_summary_df

In [5]:
challenge_set_with_trajectory = pd.DataFrame({})

unique_dates = challenge_set['date'].unique()
for current_date in unique_dates:
    today_date = str(current_date.date())
    tomorrow_date = str((current_date + timedelta(days=1)).date())
    print(f"Prosessing flights from {today_date = } to {tomorrow_date = }")

    if today_date != '2022-12-31':
        today_trajectory_df = pq.ParquetDataset(f'../data/trajactory_files/{today_date}.parquet').read().to_pandas()
        tomorrow_trajectory_df = pq.ParquetDataset(f'../data/trajactory_files/{tomorrow_date}.parquet').read().to_pandas()
        trajectory_df = pd.concat([today_trajectory_df, tomorrow_trajectory_df], ignore_index=True)
    else:
        trajectory_df = pq.ParquetDataset(f'../data/trajactory_files/{today_date}.parquet').read().to_pandas()
        
    summarized_trajectory_df = summarize_trajectories(trajectory_df=trajectory_df)
    
    filtered_df = challenge_set[challenge_set['date'] == current_date]

    merged_df = pd.merge(filtered_df, summarized_trajectory_df, on='flight_id', how='left')
    
    challenge_set_with_trajectory = pd.concat([challenge_set_with_trajectory, merged_df], ignore_index=False)
        
display(challenge_set_with_trajectory)

Prosessing flights from today_date = '2022-01-01' to tomorrow_date = '2022-01-02'
Prosessing flights from today_date = '2022-01-02' to tomorrow_date = '2022-01-03'
Prosessing flights from today_date = '2022-01-03' to tomorrow_date = '2022-01-04'
Prosessing flights from today_date = '2022-01-04' to tomorrow_date = '2022-01-05'
Prosessing flights from today_date = '2022-01-05' to tomorrow_date = '2022-01-06'
Prosessing flights from today_date = '2022-01-06' to tomorrow_date = '2022-01-07'
Prosessing flights from today_date = '2022-01-07' to tomorrow_date = '2022-01-08'
Prosessing flights from today_date = '2022-01-08' to tomorrow_date = '2022-01-09'
Prosessing flights from today_date = '2022-01-09' to tomorrow_date = '2022-01-10'
Prosessing flights from today_date = '2022-01-10' to tomorrow_date = '2022-01-11'
Prosessing flights from today_date = '2022-01-11' to tomorrow_date = '2022-01-12'
Prosessing flights from today_date = '2022-01-12' to tomorrow_date = '2022-01-13'
Prosessing fligh

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,...,longitude_median,altitude_mean,groundspeed_mean,track_mean,vertical_rate_mean,track_unwrapped_mean,u_component_of_wind_mean,v_component_of_wind_mean,temperature_mean,specific_humidity_mean
0,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01T13:46:00Z,...,-4.895513,20996.873271,317.161594,266.904445,12.768124,266.904445,31.354330,28.956784,244.875034,0.001527
1,248760618,2022-01-01,f6f610e73002b8892a239a81321f7f1d,LEBL,Barcelona,ES,KMIA,Miami,US,2022-01-01T09:55:00Z,...,-10.289873,30316.293173,425.555502,253.416154,66.719357,254.283623,21.483643,-8.009300,227.431051,0.000523
2,248753824,2022-01-01,139670936660762c230ca92556ba842b,ESSA,Stockholm Arlanda,SE,KORD,Chicago O'Hare,US,2022-01-01T09:39:00Z,...,0.848592,28087.460730,346.017174,234.214404,-35.945267,226.498682,28.988018,5.062526,225.055003,0.000467
3,248753852,2022-01-01,509dc61bb54fbab0e5406067c95603e2,LSZH,Zurich,CH,KPHL,Philadelphia,US,2022-01-01T11:04:00Z,...,-58.456228,31606.355298,405.077422,260.254022,80.900909,177.655929,32.200133,2.404330,227.891101,0.000931
4,248755934,2022-01-01,d0610d000dcf26b1d7bba8103ecc393d,EIDW,Dublin,IE,EGLL,London Heathrow,GB,2022-01-01T12:36:00Z,...,-6.001753,14895.710572,354.972270,131.665419,2754.662045,131.665419,22.697229,20.421709,255.205584,0.001545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,258058361,2022-12-31,85ee68e5b7b5acf24ba00d1318eca1e8,LFPG,Paris Charles de Gaulle,FR,KMIA,Miami,US,2022-12-31T09:38:00Z,...,-6.812298,27520.238596,352.816281,258.075719,105.885193,258.075719,41.300959,34.816118,230.600595,0.001169
283,258071247,2022-12-31,570cf7d5ebbd691bcba63e7466607da7,LTFM,iGA Istanbul,TR,EDDB,Berlin Brandenburg,DE,2022-12-31T09:27:00Z,...,19.895807,31307.635402,358.665526,312.942321,-74.184695,312.942321,16.449682,-17.756664,222.159498,0.000440
284,258059152,2022-12-31,5a7e43e4f981539ae3d3b1cb31591b7c,EDDL,Dusseldorf,DE,EIDW,Dublin,IE,2022-12-31T09:52:00Z,...,-1.753693,27064.704323,306.001767,278.353987,-246.922841,279.965484,48.607200,26.609141,229.831493,0.001106
285,258072276,2022-12-31,a1c078516f9f9e90cacec61854cad45b,LFPG,Paris Charles de Gaulle,FR,EIDW,Dublin,IE,2022-12-31T09:37:00Z,...,-2.913676,29506.571457,302.045249,299.062133,-271.516294,299.062133,45.752148,27.416125,225.945456,0.000713


In [6]:
def save_dataframe(path: str, dataframe: pd.DataFrame) -> None:
    dataframe.to_csv(path, index=False)
    print(f"{path} is saved!")

In [7]:
save_dataframe(path="../challenge_set_with_trajectory.csv", dataframe=challenge_set_with_trajectory)

../challenge_set_with_trajectory.csv is saved!


In [8]:
submission_set = pd.read_csv('../submission_set.csv')

submission_set['date'] = pd.to_datetime(submission_set['date'])

display(submission_set)

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,arrival_time,aircraft_type,wtc,airline,flight_duration,taxiout_time,flown_distance,tow
0,248753821,2022-01-01,3b3de0f3ad0ee192513995c02f7bf7cf,LTFJ,Istanbul Sabiha Gokcen,TR,LFLL,Lyon,FR,2022-01-01T09:44:00Z,2022-01-01T12:48:33Z,B738,M,6351ec1b849adacc0cbb3b1313d8d39b,170,15,1122,NaN
1,248753822,2022-01-01,e06dd03d4a879ca37d9e18c1bd7cad16,EBBR,Brussels,BE,KJFK,New York JFK,US,2022-01-01T09:45:00Z,2022-01-01T17:49:51Z,A333,H,bdeeef3a675587d530de70a25d7118d2,470,15,3205,NaN
2,248754498,2022-01-01,2d3b1c962c78c4ebeef11bcd51b9e94c,KMIA,Miami,US,EGLL,London Heathrow,GB,2022-01-01T01:52:00Z,2022-01-01T09:55:16Z,B77W,H,5543e4dc327359ffaf5b9c0e6faaf0e1,473,10,3965,NaN
3,248757623,2022-01-01,81564432d3ee97c4bdf4cd8f006753dc,EGCN,Doncaster Sheffield,GB,LEAL,Alicante,ES,2022-01-01T08:20:00Z,2022-01-01T11:06:08Z,B38M,M,3922524069809ac4326134429751e26f,156,10,986,NaN
4,248763603,2022-01-01,84be079d7e660db105d91f600b4b3d59,EIDW,Dublin,IE,LFLL,Lyon,FR,2022-01-01T11:01:00Z,2022-01-01T13:00:43Z,A320,M,a73f82288988b79be490c6322f4c32ed,105,15,686,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105954,258066302,2022-12-31,2d3b4446c4d05a25196a9d52cab936fb,LTFJ,Istanbul Sabiha Gokcen,TR,EKCH,Copenhagen,DK,2022-12-31T09:36:00Z,2022-12-31T13:12:17Z,B38M,M,6351ec1b849adacc0cbb3b1313d8d39b,201,15,1199,NaN
105955,258068609,2022-12-31,253fd692ed441fac523081471c067772,LOWW,Vienna,AT,KIAD,Washington Dulles,US,2022-12-31T09:49:00Z,2022-12-31T19:38:26Z,B763,H,5d407cb11cc29578cc3e292e743f5393,575,14,3937,NaN
105956,258068876,2022-12-31,c9fca302ca2e28acab0eb0bb1b46f11b,LTFM,iGA Istanbul,TR,LSZH,Zurich,CH,2022-12-31T09:25:00Z,2022-12-31T12:24:24Z,A321,M,6351ec1b849adacc0cbb3b1313d8d39b,154,25,988,NaN
105957,258064675,2022-12-31,00f96ad0e382476649574ba044c764fc,EHAM,Amsterdam,NL,EDDF,Frankfurt,DE,2022-12-31T10:04:21Z,2022-12-31T10:55:35Z,A320,M,f502877cab405652cf0dd70c2213e730,42,9,240,NaN


In [ ]:
submission_set_with_trajectory = pd.DataFrame({})

unique_dates = submission_set['date'].unique()
for current_date in unique_dates:
    today_date = str(current_date.date())
    tomorrow_date = str((current_date + timedelta(days=1)).date())
    print(f"Prosessing flights from {today_date = } to {tomorrow_date = }")

    if today_date != '2022-12-31':
        today_trajectory_df = pq.ParquetDataset(f'../data/trajactory_files/{today_date}.parquet').read().to_pandas()
        tomorrow_trajectory_df = pq.ParquetDataset(f'../data/trajactory_files/{tomorrow_date}.parquet').read().to_pandas()
        trajectory_df = pd.concat([today_trajectory_df, tomorrow_trajectory_df], ignore_index=True)
    else:
        trajectory_df = pq.ParquetDataset(f'../data/trajactory_files/{today_date}.parquet').read().to_pandas()
        
    summarized_trajectory_df = summarize_trajectories(trajectory_df=trajectory_df)
    
    filtered_df = submission_set[submission_set['date'] == current_date]

    merged_df = pd.merge(filtered_df, summarized_trajectory_df, on='flight_id', how='left')
    
    submission_set_with_trajectory = pd.concat([submission_set_with_trajectory, merged_df], ignore_index=False)
        
display(submission_set_with_trajectory)

Prosessing flights from today_date = '2022-01-01' to tomorrow_date = '2022-01-02'
Prosessing flights from today_date = '2022-01-02' to tomorrow_date = '2022-01-03'
Prosessing flights from today_date = '2022-01-03' to tomorrow_date = '2022-01-04'
Prosessing flights from today_date = '2022-01-04' to tomorrow_date = '2022-01-05'
Prosessing flights from today_date = '2022-01-05' to tomorrow_date = '2022-01-06'
Prosessing flights from today_date = '2022-01-06' to tomorrow_date = '2022-01-07'
Prosessing flights from today_date = '2022-01-07' to tomorrow_date = '2022-01-08'
Prosessing flights from today_date = '2022-01-08' to tomorrow_date = '2022-01-09'
Prosessing flights from today_date = '2022-01-09' to tomorrow_date = '2022-01-10'
Prosessing flights from today_date = '2022-01-10' to tomorrow_date = '2022-01-11'
Prosessing flights from today_date = '2022-01-11' to tomorrow_date = '2022-01-12'
Prosessing flights from today_date = '2022-01-12' to tomorrow_date = '2022-01-13'
Prosessing fligh

In [ ]:
save_dataframe(path="../submission_set_with_trajectory.csv", dataframe=submission_set_with_trajectory)